# Обнаружение статистически значимых отличий в уровнях экспрессии генов больных раком

Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). Определяется она следующим образом:

$Fc(C,T)= (T/C, где T>C) и (−C/T где  T<C)$

$C,T$ — средние значения экспрессии гена в control и treatment группах соответственно. По сути, fold change показывает, во сколько раз отличаются средние двух выборок.

In [1]:
import pandas as pd
import numpy as np

from scipy.stats import wilcoxon
from statsmodels.sandbox.stats.multicomp import multipletests 

In [2]:
import scipy
print(np.__version__)
print(pd.__version__)
print(scipy.__version__)

1.16.2
0.24.2
1.2.1


#### Инструкции к решению задачи
Задание состоит из трёх частей. Если не сказано обратное, то уровень значимости нужно принять равным 0.05.

#### Часть 1: применение t-критерия Стьюдента
В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

1. для групп normal (control) и early neoplasia (treatment)
2. для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.

In [3]:
data = pd.read_csv('gene_high_throughput_sequencing.csv', sep = ',', header = 0)

— проверяем данные на нормальность;
— проверяем отличия с помощью статистического теста;
— доверительным интервалом оцениваем масштаб (среднее при нормальности или медиана для ненормальности данных);

In [4]:
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [5]:
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Columns: 15750 entries, Patient_id to EIF1AY
dtypes: float64(15748), object(2)
memory usage: 8.7+ MB


In [6]:
diagnosis = data['Diagnosis'].unique()
columns = data.columns[2:]
print diagnosis, columns[0]

['normal' 'early neoplasia' 'cancer'] LOC643837


In [7]:
pairs = [['normal', 'early neoplasia'], ['early neoplasia', 'cancer']] 

In [8]:
from scipy import stats

In [21]:
%%time
for one in diagnosis:
    ppp= []
    for i in range(len(columns)):
        statistic, pvalue = stats.shapiro(data.loc[data.Diagnosis == one, columns[i]])
        ppp.append(pvalue)
    _, p_corr_fdr_bh, _, _ = multipletests(ppp, method='fdr_bh')
    print one, np.mean(p_corr_fdr_bh)
        

normal 0.48916447738314883
early neoplasia 0.5025248753479753
cancer 0.45944759206232083
CPU times: user 2min 57s, sys: 877 ms, total: 2min 58s
Wall time: 2min 59s


полученные значения существенно больше 0.05. данные на нормальность проверены. можно применять критерий Стьюдента

In [14]:
%%time
n_en = []
en_c = []
for pair in pairs:
    n = 0
    for i in range(len(columns)):
        stat, p_value = stats.ttest_ind(data.loc[data.Diagnosis == pair[0], columns[i]], 
                            data.loc[data.Diagnosis == pair[1], columns[i]], 
                            equal_var = False)
        # нужно обратить нимание, что equal_var = False не предполагает одинаков дисп 
        if p_value < 0.05:
            n += 1
        if pair == pairs[0]:
            n_en.append(p_value)
        if pair == pairs[1]:
            en_c.append(p_value)
            
    print pair, n

['normal', 'early neoplasia'] 1575
['early neoplasia', 'cancer'] 3490
CPU times: user 3min 45s, sys: 1 s, total: 3min 46s
Wall time: 3min 46s


In [15]:
with open('c4_w4_1.txt', 'w') as answer:
    answer.write(str(1575))
with open('c4_w4_2.txt', 'w') as answer:
    answer.write(str(3490))

In [16]:
import pickle
# define a list of places
with open('n_en_p_values.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(n_en, filehandle)
with open('en_c_p_values.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(en_c, filehandle)

#### Часть 2: поправка методом Холма

Для этой части задания вам понадобится модуль multitest из statsmodels.

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться $поправкой$ $Бонферрони$, то есть использовать уровень значимости $0.05/2$ вместо $0.05$ для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция $Холма-Бонферрони$. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения $fold$ $change$ и выпишите в ответ число таких значимых изменений, абсолютное значение $fold$ $change$ которых больше, чем $1.5$.

Обратите внимание, что применять поправку на множественную проверку нужно ко всем значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия.
при использовании поправки на уровне значимости $0.025$ меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом $0.025$, а не $0.05$)!

In [17]:
import statsmodels.stats.multitest as smm

1. if treatment > control => fold_change = treatment / control
2. if treatment < control => fold_change = -(control / treatment) 
3. normal (control) & early neoplasia (treatment)
4. early neoplasia (control) & cancer (treatment)
5. fold_change > 1.5

In [5]:
import pickle
with open('n_en_p_values.data', 'rb') as filehandle:
    # read the data as binary data stream
    n_en_p_values = pickle.load(filehandle)
with open('en_c_p_values.data', 'rb') as filehandle:
    # read the data as binary data stream
    en_c_p_values = pickle.load(filehandle)

In [28]:
# коррекция Холма
reject, pvals_corrected_holm_n_en, alphacSidak, alphacBonf = multipletests(n_en_p_values, method='holm')
reject, pvals_corrected_holm_en_c, alphacSidak, alphacBonf = multipletests(en_c_p_values, method='holm')

# коррекция Бонферрони
p_corr = np.array([pvals_corrected_holm_n_en, pvals_corrected_holm_en_c])
reject, p_corr_bonf, alphacSidak, alphacBonf = multipletests(p_corr, is_sorted=True, method='bonferroni')

columns = data.columns[2:]
pairs = [['normal', 'early neoplasia'], ['early neoplasia', 'cancer']]

for j in range(2):
    n = 0
    for i in range(len(p_corr_bonf[j])):
        if p_corr_bonf[j][i] < 0.05:
            control = data.loc[data.Diagnosis == pairs[j][0], columns[i]].mean()
            treatment = data.loc[data.Diagnosis == pairs[j][1], columns[i]].mean()
            if treatment > control:
                if treatment / control > 1.5:
                    n += 1
            elif treatment < control:
                if control / treatment > 1.5:
                    n += 1
    
    print pairs[j], n
    

['normal', 'early neoplasia'] 2
['early neoplasia', 'cancer'] 77


In [26]:
with open('c4_w4_3.txt', 'w') as answer:
    answer.write('2')
with open('c4_w4_4.txt', 'w') as answer:
    answer.write('77')

#### Часть 3: поправка методом Бенджамини-Хохберга

Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от $H_0$ , когда они есть, и будут чаще отклонять $H_0$ , когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых $abs(fold change) > 1.5$.

In [30]:
# коррекция Бенджамини-Хохберга
reject, pvals_corrected_bh_n_en, alphacSidak, alphacBonf = multipletests(n_en_p_values, method='fdr_bh')
reject, pvals_corrected_bh_en_c, alphacSidak, alphacBonf = multipletests(en_c_p_values, method='fdr_bh')

# коррекция Бонферрони
p_corr = np.array([pvals_corrected_bh_n_en, pvals_corrected_bh_en_c])
reject, p_corr_bonf, alphacSidak, alphacBonf = multipletests(p_corr, is_sorted=True, method='bonferroni')

columns = data.columns[2:]
pairs = [['normal', 'early neoplasia'], ['early neoplasia', 'cancer']]

for j in range(2):
    n = 0
    for i in range(len(p_corr_bonf[j])):
        if p_corr_bonf[j][i] < 0.05:
            control = data.loc[data.Diagnosis == pairs[j][0], columns[i]].mean()
            treatment = data.loc[data.Diagnosis == pairs[j][1], columns[i]].mean()
            if treatment > control:
                if treatment / control > 1.5:
                    n += 1
            elif treatment < control:
                if control / treatment > 1.5:
                    n += 1
    
    print pairs[j], n
    

['normal', 'early neoplasia'] 4
['early neoplasia', 'cancer'] 524


In [31]:
with open('c4_w4_5.txt', 'w') as answer:
    answer.write('4')
with open('c4_w4_6.txt', 'w') as answer:
    answer.write('524')